<a href="https://www.kaggle.com/code/davidphummel/computer-vision-is-it-a-dog-or-a-cat?scriptVersionId=196908614" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Computer Vision - Is it a dog or a cat?

This is the first Juypter notebook I created while reading chapter 1 of the Fastai course called [Practical Deep Learning](https://course.fast.ai).  Practical Deep Learning is a free course designed for people who have some coding experience and want to learn how to apply deep learning and maching learning to practical problems.  

I have found an understanding of how to program in Python is a requirement to complete this course along with a good understanding of the mathematical concepts.  I have spent some time refershing my memory of the NumPy package using the book [NumPy for Data Analysis](https://wesmckinney.com/book/numpy-basics) by Wes McKinney.  The NumPy pachage is used in the course to load and clean data along with building the models.  I have also found the need to refresh my memory of some of mathmatical concepts included in deep learning including gradient descent.  I really recommend the videos produced by [StatQuest](https://statquest.org/) on [gradient descent](https://youtu.be/sDv4f4s2SB8?si=SZjknb9ca-nl18ie) and others related to statistics and machine learning.  Josh Starmer created StatQuest to attempt to explain statistics to his former co-wokers.  His vidoes are simple easy to understand, and engaging! 

This Juypter notebook is a copy of Jeremy Howard's notebook called [Is it a bird? Creating a model from your own data](https://www.kaggle.com/code/jhoward/is-it-a-bird-creating-a-model-from-your-own-data).  I modified Jeremy's notebook to determine if an image is a dog or a cat.  I have two portugese water dogs and 1 three legged cat, so I thought it would be fun to focus on them.  The objective of the notebook is to create and train a model to determine if an image is a dog or cat.

In [2]:
# It's a good idea to ensure you're running the latest version of any libraries you need.
# `!pip install -Uqq <libraries>` upgrades to the latest version of <libraries>
# You can safely ignore any warnings or errors pip spits out about running as root or incompatibilities
#import os

iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
if iskaggle:
    !pip install -Uqq fastai fastbook

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-io 0.21.0 requires tensorflow-io-gcs-filesystem==0.21.0, which is not installed.
explainable-ai-sdk 1.3.2 requires xai-image-widget, which is not installed.
tensorflow 2.6.2 requires numpy~=1.19.2, but you have numpy 1.20.3 which is incompatible.
tensorflow 2.6.2 requires six~=1.15.0, but you have six 1.16.0 which is incompatible.
tensorflow 2.6.2 requires typing-extensions~=3.7.4, but you have typing-extensions 3.10.0.2 which is incompatible.
tensorflow 2.6.2 requires wrapt~=1.12.1, but you have wrapt 1.13.3 which is incompatible.
tensorflow-transform 1.5.0 requires absl-py<0.13,>=0.9, but you have absl-py 0.15.0 which is incompatible.
tensorflow-transform 1.5.0 requires numpy<1.20,>=1.16, but you have numpy 1.20.3 which is incompatible.
tensorflow-transform 1.5.0 requires pyarrow<6,>=1, but you have p

The basic steps we'll take are:

1. Use DuckDuckGo to find images of dogs and cats on the Internet.
2. Fine-tune a pretrained neural network to recognise these two animals.
3. Try running this model on set of pictures and see if it works.

## Step 1: Download images of cats and dogs

Fastcore uses this flexibility to add to Python features inspired by other languages like multiple dispatch from Julia, mixins from Ruby, and currying, binding, and more from Haskell. It also adds some “missing features” and clean up some rough edges in the Python standard library, such as simplifying parallel processing, and bringing ideas from NumPy over to Python’s list type.

Let's start out by defining a function to search the Internet using DuckDuckGo for a term (e.g., "dog") and return a set of URLs to images of the term.

In [4]:
from fastcore.all import *
from fastbook import search_images_ddg

def search_images(term, max=30):
  print(f"Searching for '{term}' using DuckDuckGo")
  return search_images_ddg(term, max_images=max)

AttributeError: 'str' object has no attribute '__name__'

Let's search for a dog, cat, and bird image and see what kind of result we get. I am downloading bird image because I want to see how the model reacts to a non-dog/non-cat image.  

We'll start by getting URLs using the search function.

In [ ]:
from fastdownload import download_url
from fastai.vision.all import *
from PIL import Image, ImageOps

# Download the image of Max (see my-family-dogs dataset)
max_file = "../input/my-family-pets/Max.jpg"
max_image = Image.open(max_file)
max_image = ImageOps.exif_transpose(max_image)
max_image.to_thumb(256, 256)

In [ ]:
# Download the first image of Lucy (see My Family Pets dataset)
lucy_file = "../input/my-family-pets/Lucy.jpg"
lucy_image = Image.open(lucy_file)
lucy_image = ImageOps.exif_transpose(lucy_image)
lucy_image.to_thumb(256, 256)

In [ ]:
# Download the second image of Lucy (see My Family Pets dataset)
lucy_file2 = "../input/my-family-pets/Lucy_2.jpg"
lucy_image2 = Image.open(lucy_file2)
lucy_image2 = ImageOps.exif_transpose(lucy_image2)
lucy_image2.to_thumb(256, 256)

In [ ]:
# Download the third image of Lucy (see My Family Pets dataset)
lucy_file3 = "../input/my-family-pets/Lucy_3.jpg"
lucy_image3 = Image.open(lucy_file3)
lucy_image3 = ImageOps.exif_transpose(lucy_image3)
lucy_image3.to_thumb(256, 256)

In [ ]:
# Download image of Chica (see My Family Pets dataset)
chica_file = '../input/my-family-pets/Chica.jpg'
chica_image = Image.open(chica_file)
chica_image = ImageOps.exif_transpose(chica_image)
chica_image.to_thumb(256, 256)

The searches seem to be giving reasonable results, so let's grab a few examples dog and cat photos, and save each group of photos to a different folder.  We are also grabbing a set of images with different lighting conditions.

In [ ]:
from time import sleep

terms = 'dog','cat'
path = Path('animals')

for term in terms:
    dest = (path/term)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest, urls=search_images(f'{term} photo'))
    sleep(10)  # Pause between searches to avoid over-loading server
    download_images(dest, urls=search_images(f'{term} sun photo'))
    sleep(10)
    download_images(dest, urls=search_images(f'{term} shade photo'))
    sleep(10)
    resize_images(path/term, max_size=400, dest=path/term)

## Step 2: Train the model

Some photos might not download correctly which could cause our model training to fail, so we'll remove them:

In [ ]:
# Find the images that can't be opened
failed = verify_images(get_image_files(path))

# Remove the image files which can't be openend.  The map function will execute the
# Path.unlink function over the failed images in the failed (L) object.
failed.map(Path.unlink)

print(f"{len(failed)} images were removed because they could not be verified.")

To train a model, we'll need `DataLoaders`, which is an object that contains a *training set* (the images used to create a model) and a *validation set* (the images used to check the accuracy of a model -- not used during training). In `fastai` we can create that easily using a `DataBlock`, and view sample images from it:

Here what each of the `DataBlock` parameters means:

- ***blocks=(ImageBlock, CategoryBlock)*** - the input to our model are images and the outputs are catogries (in this case either a dog or a cat)

- ***get_items=get_image_files*** - to find all the inputs to the model we run the 'get_image_files' function which results a list of all the image files in the path.

- ***splitter=RandomSplitter(valid_pct=0.2, seed=42)*** - spilt the data into training and validation sets randomly using 20% of the adta for the valication set.

- ***get_y=parent_label*** - the labels ('y' value) is the name of the 'parent' of each file (i.e., the name of the folder each file is located in which will be *dog* or *cat*.

- ***item_tfms=[Resize(192, method='squish')]*** - resize each image to 192x192 pixles by "squishing" (vs. cropping) before training.

In [ ]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path, bs=32)

dls.show_batch(max_n=6)

Now we're ready to train our model. The fastest widely used computer vision model is `resnet18`. You can train this in a few minutes, even on a CPU! (On a GPU, it generally takes under 10 seconds...)

`fastai` comes with a helpful `fine_tune()` method which automatically uses best practices for fine tuning a pre-trained model, so we'll use that.

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(3)

Generally when I run this I see 100% accuracy on the validation set (although it might vary a bit from run to run).

"Fine-tuning" a model means that we're starting with a model someone else has trained using some other dataset (called the *pretrained model*), and adjusting the weights a little bit so that the model learns to recognise your particular dataset. In this case, the pretrained model was trained to recognise photos in *imagenet*, and widely-used computer vision dataset with images covering 1000 categories) For details on fine-tuning and why it's important, check out the [free fast.ai course](https://course.fast.ai/).

## Step 3: Use the model

Let's see what our model thinks about that animales downloaded at the start:

In [ ]:
# Test the model using Max
result,_,prob = learn.predict(max_image)
display(max_image.to_thumb(256, 256))
print(f"The model thinks Max is a {result}.")
print(f"Dog Probability: {prob[1]:.4f}")
print(f"Cat Probability: {prob[0]:.4f}")

# Test the model using Lucy's first image
result,_,prob = learn.predict(lucy_image)
display(lucy_image.to_thumb(256, 256))
print(f"The model thinks Lucy a {result}.")
print(f"Dog Probability: {prob[1]:.4f}")
print(f"Cat Probability: {prob[0]:.4f}")

# Test the model using Lucy's second image
result,_,prob = learn.predict(lucy_image2)
display(lucy_image2.to_thumb(256, 256))
print(f"The model thinks Lucy is a {result}.")
print(f"Dog Probability: {prob[1]:.4f}")
print(f"Cat Probability: {prob[0]:.4f}")

# Test the model using Lucy's third image
result,_,prob = learn.predict(lucy_image3)
display(lucy_image3.to_thumb(256, 256))
print(f"The model thinks Lucy is a {result}.")
print(f"Dog Probability: {prob[1]:.4f}")
print(f"Cat Probability: {prob[0]:.4f}")

# Test the model using Chica
result,_,prob = learn.predict(PILImage.create(chica_image))
display(chica_image.to_thumb(256, 256))
print(f"The model things Chica is a {result}.")
print(f"Dog Probability: {prob[1]:.4f}")
print(f"Cat Probability: {prob[0]:.4f}")

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()
interp.plot_top_losses(5,nrows=1)

Good job, resnet18. :)

So, as you see, in the space of a few years, creating computer vision classification models has gone from "so hard it's a joke" to "trivially easy and free"!

It's not just in computer vision. Thanks to deep learning, computers can now do many things which seemed impossible just a few years ago, including [creating amazing artworks](https://openai.com/dall-e-2/), and [explaining jokes](https://www.datanami.com/2022/04/22/googles-massive-new-language-model-can-explain-jokes/). It's moving so fast that even experts in the field have trouble predicting how it's going to impact society in the coming years.

One thing is clear -- it's important that we all do our best to understand this technology, because otherwise we'll get left behind!

Now it's your turn. Click "Copy & Edit" and try creating your own image classifier using your own image searches!

If you enjoyed this, please consider clicking the "upvote" button in the top-right -- it's very encouraging to us notebook authors to know when people appreciate our work.